In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import all relevant packages

import os,ast
import shutil
import pandas as pd
import numpy as np
from sklearn import model_selection
from tqdm import tqdm

In [22]:
# Create a path to your dataset

main_dir = r"/content/drive/MyDrive/DATA_FOR_COLAB/yolov5"

**Visualise your dataset with the information you have**

In [4]:
df = pd.read_csv(os.path.join(main_dir,"train.csv"))
sample_df = pd.read_csv(os.path.join(main_dir,"sample_submission.csv"))

In [5]:
df

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1
...,...,...,...,...,...
147788,5e0747034,1024,1024,"[64.0, 619.0, 84.0, 95.0]",arvalis_2
147789,5e0747034,1024,1024,"[292.0, 549.0, 107.0, 82.0]",arvalis_2
147790,5e0747034,1024,1024,"[134.0, 228.0, 141.0, 71.0]",arvalis_2
147791,5e0747034,1024,1024,"[430.0, 13.0, 184.0, 79.0]",arvalis_2


In [6]:
sample_df

,image_id,PredictionString
0,aac893a91,1.0 0 0 50 50
1,51f1be19e,1.0 0 0 50 50
2,f5a1f0358,1.0 0 0 50 50
3,796707dd7,1.0 0 0 50 50
4,51b3e36ab,1.0 0 0 50 50
5,348a992bb,1.0 0 0 50 50
6,cc3532ff6,1.0 0 0 50 50
7,2fd875eaa,1.0 0 0 50 50
8,cb8d261a3,1.0 0 0 50 50
9,53f253011,1.0 0 0 50 50


In [7]:
df.bbox = df.bbox.apply(ast.literal_eval)

In [8]:
df = df.groupby("image_id")["bbox"].apply(list).reset_index(name="all_bbox")

In [9]:
df

,image_id,all_bbox
0,00333207f,"[[0, 654, 37, 111], [0, 817, 135, 98], [0, 192..."
1,005b0d8bb,"[[765.0, 879.0, 116.0, 79.0], [84.0, 539.0, 15..."
2,006a994f7,"[[437.0, 988.0, 98.0, 36.0], [309.0, 527.0, 11..."
3,00764ad5d,"[[89.0, 256.0, 113.0, 107.0], [216.0, 282.0, 1..."
4,00b5fefed,"[[709.0, 97.0, 204.0, 105.0], [775.0, 250.0, 1..."
...,...,...
3368,ffb445410,"[[0.0, 534.0, 54.0, 118.0], [0.0, 480.0, 38.0,..."
3369,ffbf75e5b,"[[0, 697, 21, 58], [104, 750, 77, 75], [65, 84..."
3370,ffbfe7cc0,"[[256.0, 0.0, 64.0, 99.0], [390.0, 0.0, 48.0, ..."
3371,ffc870198,"[[447.0, 976.0, 78.0, 48.0], [18.0, 141.0, 218..."


In [10]:
data_train,data_valid = model_selection.train_test_split(df,test_size=0.1,random_state=40,shuffle=True)

In [11]:
data_train = data_train.reset_index(drop=True)
data_valid = data_valid.reset_index(drop=True)

**Create the required directiories and sub folders**

In [12]:
%cd /content/drive/MyDrive/DATA_FOR_COLAB/yolov5
!mkdir wheat_head_data
!mkdir wheat_head_data/images
!mkdir wheat_head_data/images/train
!mkdir wheat_head_data/images/validation
!mkdir wheat_head_data/labels
!mkdir wheat_head_data/labels/train
!mkdir wheat_head_data/labels/validation

/content/drive/MyDrive/DATA_FOR_COLAB/yolov5
mkdir: cannot create directory ‘wheat_head_data’: File exists
mkdir: cannot create directory ‘wheat_head_data/images’: File exists
mkdir: cannot create directory ‘wheat_head_data/images/train’: File exists
mkdir: cannot create directory ‘wheat_head_data/images/validation’: File exists
mkdir: cannot create directory ‘wheat_head_data/labels’: File exists
mkdir: cannot create directory ‘wheat_head_data/labels/train’: File exists
mkdir: cannot create directory ‘wheat_head_data/labels/validation’: File exists


In [13]:
output_dir = r"/content/drive/MyDrive/DATA_FOR_COLAB/yolov5/wheat_head_data"

In [14]:
def check_data(data,d_type="train"):
    for row in tqdm(data.iterrows(),total = len(data)):
        img = row[1]["image_id"]
        boundings = row[1]["all_bbox"]
        yolo_dt = []
        for bbox in boundings:
            x = bbox[0]
            y = bbox[1]
            w = bbox[2]
            h = bbox[3]
            x_center = (x+w)/2
            y_center = (y+h)/2
            x_center /= 1024.0
            y_center /= 1024.0
            w /= 1024.0
            h /= 1024.0
            yolo_dt.append([0,x_center,y_center,w,h])
        yolo_dt = np.array(yolo_dt)
        np.savetxt(os.path.join(output_dir,f"labels/{d_type}/{img}.txt"),yolo_dt,fmt = ["%d","%f","%f","%f","%f"])
        shutil.copyfile(os.path.join(main_dir,f"train/{img}.jpg"),os.path.join(output_dir,f"images/{d_type}/{img}.jpg"))

In [ ]:
check_data(data_train,d_type="train")
check_data(data_valid,d_type="validation")

100%|██████████| 338/338 [03:18<00:00,  1.70it/s]


**Train your data by defining suitable no. of batches and epochs**
- ```img```: define input image size
- ```batch```: determine batch size
- ```epochs```: define the number of training epochs. (Note: often, 3000+ are common here!)
- ```data```: set the path to our yaml file
- ```cfg```: specify our model configuration
- ```weights```: specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive folder)
- ```name```: result names
- ```nosave```: only save the final checkpoint
- ```cache```: cache images for faster training

In [18]:
!python train.py --img 1024 --batch 2 --epochs 10 --data wheat.yaml --cfg models/yolov5s.yaml --name wheat

train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=wheat.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=2, imgsz=1024, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=wheat, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-253-ga45e472 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, trans

In [21]:
# create a prediction on validation data

# !python detect.py --weights /content/drive/MyDrive/DATA_FOR_COLAB/yolov5/runs/train/wheat5/weights/best.pt --img 1024 --conf 0.4 --source /content/drive/MyDrive/DATA_FOR_COLAB/yolov5/wheat_head_data/images/validation